# Title: py_mulval export pregraph
### Purpose: Show how to load an XSB fact file and dump a netgraph (not attack graph)
### Author: @mjz

## Global imports and setup here

In [2]:

import logging
import os
import sys
import uuid
import pathlib

py_mulval_path = r'/opt/projects/diss/py-mulval/src'
sys.path.append(py_mulval_path)

# Just importing for their flags right now
import py_mulval.py_mulval as py_mulval
import py_mulval.boromir as boromir
import py_mulval.log_util as log_util
import py_mulval.mulpy as mulpy
import py_mulval.metrics as secmet
import py_mulval.attack_graph as attack_graph

from py_mulval import flags
FLAGS = flags.FLAGS
FLAGS([sys.argv[0]]) # dont expect cli args from jupyter

['/opt/projects/diss/jupyter_nbs/py36/lib/python3.6/site-packages/ipykernel_launcher.py']

In [3]:
print(FLAGS.flags_into_string())

--noonly_check_args
--nopdb_post_mortem
--norun_with_pdb
--norun_with_profiling
--use_cprofile_for_profiling
--noalsologtostderr
--log_dir=
--nologtostderr
--showprefixforinfo
--stderrthreshold=fatal
--verbosity=-1
--num_benchmark_copies=1
--benchmark_compatibility_checking=strict
--cloud=GCP
--after_prepare_sleep_time=0
--after_run_sleep_time=0
--nobefore_cleanup_pause
--benchmarks=standard_set
--noboot_samples
--boromir_run_count=1
--nocreate_failed_run_samples
--nocreate_started_run_sample
--nodisable_interrupt_moderation
--nodisable_rss
--nodry_run
--extra_zones=
--failed_run_samples_error_length=10240
--file_log_level=debug
--ftp_proxy=
--helpmatch=
--helpmatchmd=
--http_proxy=
--https_proxy=
--noignore_package_requirements
--nolog_dmesg
--log_level=info
--num_vms=1
--owner=toor
--persistent_timeout_minutes=240
--nopublish_after_run
--norandomize_run_order
--record_lscpu
--record_proccpu
--run_stage_iterations=1
--run_stage_retries=0
--run_stage_time=0
--ssh_options=
--nostop_afte

The MulVal wrapper we developed preserves the same flags as the original and adds some new features too.

## MulVal instrumented example end-2-end

`run.py` simply calls `mulpy.py` Main() method here and forwards flags. 

In [4]:
%%bash
/opt/projects/diss/py-mulval/src/run.py --input_file=single_host_1.P -r local_exploit_rules.P --models_dir=/opt/projects/diss/py-mulval/data/models --rules_dir=/opt/projects/diss/py-mulval/data/rules --data_dir=/opt/projects/diss/py-mulval/data

2020-05-06 23:24:58,260 [MainThread  ] [INFO ] Basic Logging configured.
2020-05-06 23:24:58,260 [MainThread  ] [DEBUG] Parsed command line flags: single_host_1.P
2020-05-06 23:24:58,260 [MainThread  ] [INFO ] Setting up Mulpy environment...
Traceback (most recent call last):
  File "/opt/projects/diss/py-mulval/src/run.py", line 8, in <module>
    sys.exit(Main())
  File "/opt/projects/diss/py-mulval/src/py_mulval/mulpy.py", line 357, in Main
    SetupMulpy()
  File "/opt/projects/diss/py-mulval/src/py_mulval/mulpy.py", line 120, in SetupMulpy
    InitRunID()
  File "/opt/projects/diss/py-mulval/src/py_mulval/mulpy.py", line 114, in InitRunID
    FLAGS.run_uri = str(uuid.uuid4())[-8:]
  File "/opt/projects/diss/jupyter_nbs/py36/lib/python3.6/site-packages/absl/flags/_flagvalues.py", line 499, in __setattr__
    return self._set_unknown_flag(name, value)
  File "/opt/projects/diss/jupyter_nbs/py36/lib/python3.6/site-packages/absl/flags/_flagvalues.py", line 375, in _set_unknown_flag
  

CalledProcessError: Command 'b'/opt/projects/diss/py-mulval/src/run.py --input_file=single_host_1.P -r local_exploit_rules.P --models_dir=/opt/projects/diss/py-mulval/data/models --rules_dir=/opt/projects/diss/py-mulval/data/rules --data_dir=/opt/projects/diss/py-mulval/data\n'' returned non-zero exit status 1.

## py_mulval constants and setup here

these are passed as flags in the library, but we set them here for the notebook env

In [4]:
SEP = os.path.sep
# needed if pyxsb can't find xsb
XSB_ARCH_DIR = '/opt/apps/xsb/XSB/config/x86_64-unknown-linux-gnu'
# MulVal Install Files
MULVALROOT = '/opt/mulval'
MULVALROOT = '/opt/projects/diss/mulval/mulval'
INTERACTIONRULES = SEP.join((MULVALROOT, 'kb/interaction_rules.P'))
INTERACTIONRULES_CVSS = SEP.join(
    (MULVALROOT, 'kb/interaction_rules_with_metrics.P'))
RULES_WITH_METRIC_ARTIFACTS = SEP.join(
    (MULVALROOT, 'kb/interaction_rules_with_metric_artifacts.P'))
ATTACK_GRAPH_BIN = SEP.join((MULVALROOT, "bin/attack_graph"))
RUNNING_RULES_NAME = 'running_rules.P'
ENV_FILE_NAME = 'environment.P'
RUN_FILE_NAME = 'run.P'
# INPUT_FILE = 'single_host_1.P'
INPUT_FILE = 'h2v1s5.P'
MODELS_DIR = '/opt/projects/diss/py-mulval/data/models'
RULES_DIR = '/opt/projects/diss/py-mulval/data/rules'


if not FLAGS.rule: FLAGS['rule'].parse('local_exploit_rules.P')
FLAGS['input_file'].parse(INPUT_FILE)
FLAGS['models_dir'].parse(MODELS_DIR)
FLAGS['rules_dir'].parse(RULES_DIR)
# FLAGS['rule'].parse(RULES_DIR)


def writeFile(file_name, file_text, mode='w+'):
  """
  w  write mode
  r  read mode
  a  append mode
  w+  create file if it doesn't exist and open it in (over)write mode
      [it overwrites the file if it already exists]
  r+  open an existing file in read+write mode
  a+  create file if it doesn't exist and open it in append mode
  """
  with open(file_name, mode) as file:
    file.write(file_text)

def _SetModelsDir():
  if not FLAGS.models_dir:
    FLAGS.models_dir = SEP.join((FLAGS.base_dir, 'models'))
  logging.info('Models directory set to: {}'.format(FLAGS.models_dir))
  if not pathlib.Path(FLAGS.models_dir).exists():
    pathlib.Path(FLAGS.models_dir).mkdir(parents=True, exist_ok=True)

def _SetOutputDir():
  if not FLAGS.output_dir:
    FLAGS.output_dir = SEP.join((FLAGS.base_dir, 'output'))
  logging.info('Output directory set to: {}'.format(FLAGS.output_dir))
  if not pathlib.Path(FLAGS.output_dir).exists():
    pathlib.Path(FLAGS.output_dir).mkdir(parents=True, exist_ok=True)

def SetupMulpy():
  """Makes a new run directory and sets up flags and vars
  """
  log_util.ConfigureBasicLogging()
  logging.info('Setting up Mulpy environment...')
  if not FLAGS.run_uri: 
        FLAGS.run_uri = str(uuid.uuid4())[-8:]
  if not FLAGS.base_dir:
    FLAGS.base_dir = SEP.join(('/tmp', 'mulpy', 'runs', FLAGS.run_uri))
#   FLAGS.base_dir = SEP.join((FLAGS.base_dir, FLAGS.run_uri))
  logging.info('Base directory set to: {}'.format(FLAGS.base_dir))
  if not pathlib.Path(FLAGS.base_dir).exists():
    pathlib.Path(FLAGS.base_dir).mkdir(parents=True, exist_ok=True)

  # setup logging once we have a run_uri and base dir
  log_util.ConfigureLogging(logging.DEBUG,
                            SEP.join((FLAGS.base_dir, 'cat-dog.log')),
                            FLAGS.run_uri)
  # try to get cli into main log... not always trustworthy
  logging.info('running command line: %s' % ' '.join(sys.argv))
  

  _SetModelsDir()
  # _SetRulesDir() # if no rules_dir just use default set
  _SetOutputDir()
  # Copy model or example into base_dir
  if pathlib.Path(SEP.join((FLAGS.models_dir, FLAGS.input_file))).exists():
    input_p = pathlib.Path(
      SEP.join((FLAGS.models_dir, FLAGS.input_file))).read_text()
  else:
    input_p = """attackerLocated(internet).
  attackGoal(execCode(workStation,_)).

  hacl(internet, webServer, tcp, 80).
  hacl(webServer, _,  _, _).
  hacl(fileServer, _, _, _).
  hacl(workStation, _, _, _).
  hacl(H,H,_,_).

  /* configuration information of fileServer */
  networkServiceInfo(fileServer, mountd, rpc, 100005, root).
  nfsExportInfo(fileServer, '/export', _anyAccess, workStation).
  nfsExportInfo(fileServer, '/export', _anyAccess, webServer).
  vulExists(fileServer, vulID, mountd).
  vulProperty(vulID, remoteExploit, privEscalation).
  localFileProtection(fileServer, root, _, _).

  /* configuration information of webServer */
  vulExists(webServer, 'CAN-2002-0392', httpd).
  vulProperty('CAN-2002-0392', remoteExploit, privEscalation).
  networkServiceInfo(webServer , httpd, tcp , 80 , apache).

  /* configuration information of workStation */
  nfsMounted(workStation, '/usr/local/share', fileServer, '/export', read).
  """
  # write input file to working directory if it doesn't exist
  outfile = SEP.join((FLAGS.base_dir, FLAGS.input_file))
  logging.debug(('creating input file: %s') % outfile)
  if not pathlib.Path(outfile).exists():
    with open(outfile, 'w+') as file:
      file.write(input_p)
    logging.debug(('creating input file: %s') % outfile)


## Step by step

In [5]:

# Setup this run
SetupMulpy()

# Add overrides to SetupMulpy() constants here.
mulval_args = {}

#####
## graph_gen.sh is the entrypoint to the original MulVal.
## Here we can pass the same flags MulVal shell controller accepted to init().
## Any unknown flags here are forwarded to subsequent modules.
####
gg = py_mulval.graph_gen(**mulval_args)

####
## graph_gen() does the things graph_gen.sh does
##  uses tmpdir/runs/<uuid> as working directory
##  writes out environment.P and run.P for the given models and rules
####
gg.graph_gen()

####
##  normally we would call runMulval here but we want to stop just before tracing
##  consulting run.P from XSB should leave a trace_output.P
##  file in working dir that gets sent to attack_graph.cpp 
####
#gg.runMulVal()


2020-04-30 19:44:51,087 [MainThread  ] [INFO ] Setting up Mulpy environment...
2020-04-30 19:44:51,090 [MainThread  ] [INFO ] Base directory set to: /tmp/mulpy/runs/1dde0947
2020-04-30 19:44:51,094 1dde0947 MainThread INFO     Verbose logging to: /tmp/mulpy/runs/1dde0947/cat-dog.log
2020-04-30 19:44:51,098 1dde0947 MainThread INFO     running command line: /opt/projects/diss/jupyter_nbs/py36/lib/python3.6/site-packages/ipykernel_launcher.py -f /home/toor/.local/share/jupyter/runtime/kernel-d6c2c7a5-9c3b-4925-a4f0-3b2ed962c204.json
2020-04-30 19:44:51,099 1dde0947 MainThread INFO     Models directory set to: /opt/projects/diss/py-mulval/data/models
2020-04-30 19:44:51,101 1dde0947 MainThread INFO     Output directory set to: /tmp/mulpy/runs/1dde0947/output
2020-04-30 19:44:51,103 1dde0947 MainThread DEBUG    creating input file: /tmp/mulpy/runs/1dde0947/h2v1s5.P
2020-04-30 19:44:51,104 1dde0947 MainThread DEBUG    creating input file: /tmp/mulpy/runs/1dde0947/h2v1s5.P
2020-04-30 19:44:5

### graph_gen.runMulVal() anatomy

In [6]:
print(FLAGS.base_dir)
print(os.getcwd()) # run XSB session from cwd
# FLAGS.rules_dir
# FLAGS.input_file
# FLAGS.models_dir
# FLAGS.rule
print(XSB_ARCH_DIR) # XSB arch setting

/tmp/mulpy/runs/1dde0947
/tmp/mulpy/runs/1dde0947
/opt/apps/xsb/XSB/config/x86_64-unknown-linux-gnu


### Start XSB session

In [7]:
from pyxsb import *

pyxsb_start_session(XSB_ARCH_DIR)
print(pyxsb_query('cwd(D).'))

listing = pyxsb_query('listing.')
print(listing)
# for item in listing:
#     print(item)

# pyxsb_end_session()


[XSBFunctor(name=ret, module=None, args=[XSBAtom(name=/tmp/mulpy/runs/1dde0947)])]
[XSBAtom(name=ret)]


### Consulting environment.P

This loads the current ruleset and stages the environment, allowing us to extract goals from the input model before running trace.

In [8]:
%%bash
cat environment.P

:-['/opt/projects/diss/mulval/mulval/lib/libmulval'].  % start base run script
:-['/opt/projects/diss/mulval/mulval/src/analyzer/translate'].
:-['/opt/projects/diss/mulval/mulval/src/analyzer/attack_trace'].
:-['/opt/projects/diss/mulval/mulval/src/analyzer/auxiliary'].

:-dynamic meta/1.

:-load_dyn('running_rules.P').

:-load_dyn('h2v1s5.P').

:-assert(traceMode(completeTrace2)).  % end base run script

% set if dynamic changes file is set (-d flag)


% set if --trim | -tr flag passed

% else set if no --trim | -tr flag passed
:-load_dyn('/opt/projects/diss/mulval/mulval/src/analyzer/advances_notrim.P'). 


% add this line if CVSS flag is not set (non-zero len) 
% @TODO should expect a string here not bool
:-assert(cvss(_, none)).

% add goal if passed as a flag


% add mulval run if we're not writing the environment program



In [9]:
pyxsb_command('[environment].')
pyxsb_command("tell('goals.txt').")
pyxsb_command('writeln("Goal:"). ')
pyxsb_command("iterate(attackGoal(G),(write(' '), write_canonical(G), nl)).")
pyxsb_command('told.')

In [10]:
%%bash
# cat goals.txt
pwd

/tmp/mulpy/runs/1dde0947


### Query XSB environment

We can query XSB directly now, but derived facts aren't available

In [ ]:

# print('--------------------')
# for row in pyxsb_query('hacl(A, B, C, D).'):
#     print(u"hacl %s %s %s %s" % (row[0], row[1], row[2], row[3]))
# print('--------------------')
# for row in pyxsb_query('vulProperty(A,B,C).'):
#     print(u"vulprop %s %s %s" % (row[0], row[1], row[2]))

# print('---------attackGoal-----------')
# print(pyxsb_query('findall(attackGoal(A), attackGoal(A), L), list_apply(L,write_clause_to_stdout).'))

# print('---------attackerLocated-----------')
# print(pyxsb_query('findall(attackerLocated(A), attackerLocated(A), L), list_apply(L,write_clause_to_stdout).'))
    
# print('---------vulProperty-----------')
# print(pyxsb_query('findall(vulProperty(A,B,C),vulProperty(A,B,C),L),list_apply(L,write_clause_to_stdout).'))

# print('----------vulExists----------')
# print(pyxsb_query('findall(vulExists(A,B,C),vulExists(A,B,C),L),list_apply(L,write_clause_to_stdout).'))

# print('---------cvss-----------')
# print(pyxsb_query('findall(cvss(CVE, AC),cvss(CVE, AC),L),list_apply(L,write_clause_to_stdout).'))

# print('----------hacl----------')
# print(pyxsb_query('findall(hacl(A, B, C, D),hacl(A, B, C, D),L),list_apply(L,write_clause_to_stdout).'))

# print('---------networkServiceInfo-----------')
# print(pyxsb_query('findall(networkServiceInfo(Host, Program, someProtocol, somePort, someUser), networkServiceInfo(Host, Program, someProtocol, somePort, someUser), L), list_apply(L,write_clause_to_stdout).'))

# print('---------hasAccount-----------')
# print(pyxsb_query('findall(hasAccount(A,B,C), hasAccount(A,B,C), L), list_apply(L,write_clause_to_stdout).'))

# print('---------inCompetent-----------')
# print(pyxsb_query('findall(inCompetent(A), inCompetent(A), L), list_apply(L,write_clause_to_stdout).'))

# print('---------inSubnet-----------')
# print(pyxsb_query('findall(inSubnet(A, B)., inSubnet(A, B), L), list_apply(L,write_clause_to_stdout).'))

# print('============Derived===============')
# print('---------execCode-----------')
# print(pyxsb_query('findall(execCode(A, B)., execCode(A, B), L), list_apply(L,write_clause_to_stdout).'))


In [12]:
def queryMulval(dl_fact):
    query = 'findall({},{},L),list_apply(L,write_clause_to_stdout).'.format(dl_fact, dl_fact)
    print(query)
    result = pyxsb_query(query)
    return result
    

In [ ]:


# x = queryMulval(netaccess)
# print(queryMulval(hacl))
# print('findall(hacl(A, B, C, D),hacl(A, B, C, D),L),list_apply(L,write_clause_to_stdout).')
print(pyxsb_query('findall(hacl(A, B, C, D),hacl(A, B, C, D),L),list_apply(L,write_clause_to_stdout).'))

In [14]:
import pyxsb


# primitive
hacl = 'hacl(A, B, C, D)'

# derived 
advances = 'advances(A,B)'
netaccess = 'netAccess(A, B, C)'

x = queryMulval(hacl)


def flatten(structure, key="", path="", flattened=None):
#     print(type(structure))
    if flattened is None:
        flattened = {}
    if type(structure) not in(dict, list):
        flattened[((path + "_") if path else "") + key] = structure
    elif isinstance(structure, list):
        for i, item in enumerate(structure):
            flatten(item, "%d" % i, path + "_" + key, flattened)
    else:
        for new_key, value in structure.items():
            flatten(value, new_key, path + "_" + key, flattened)
    return flattened

# xflat = flatten(json.loads(xsb_to_json(x)))
# for (k, v) in xflat.items():
#     print(k, ': ', v)

# print(type(x))
# for item in x:
#     print(type(item))
results=[]
print('1-', type(x), len(x))
for row in x:
    xj = json.loads(xsb_to_json(row))
    print('2--', type(row), type(xj))
    for key in xj.keys():
        print('3---', type(key), key, type(xj[key]) )#, xj[key])
        
    for arg_row in xj['args']:
        print('4----',type(arg_row), arg_row) #, xj['args'][arg_key])
        if type(arg_row) is list:
            print('found results list. Size:', len(arg_row), '\nparsing results...')
            for item in arg_row: # each item is a dict of results
                print('5----',type(item), item)
                if type(item) is dict:
                    
                    for k, v in item.items():
                        
                        print('6-----', k, v)
                        if k == 'name':
                            print(v)
                        if k == 'args':
                            for items2 in v:
                                print('7------',type(item), item)
                        


# print(type(xj))
# for row in xj:
#     print('-----')
#     print(type(xj[0]), xj[0])

findall(hacl(A, B, C, D),hacl(A, B, C, D),L),list_apply(L,write_clause_to_stdout).
1- <class 'list'> 1
2-- <class 'pyxsb.XSBFunctor'> <class 'dict'>
3--- <class 'str'> pt <class 'str'>
3--- <class 'str'> name <class 'str'>
3--- <class 'str'> module <class 'NoneType'>
3--- <class 'str'> args <class 'list'>
4---- <class 'dict'> {'pt': 'variable', 'name': 'p\x05ʶ\x7f'}
4---- <class 'dict'> {'pt': 'variable', 'name': 'x\x05ʶ\x7f'}
4---- <class 'dict'> {'pt': 'variable', 'name': '\x05ʶ\x7f'}
4---- <class 'dict'> {'pt': 'variable', 'name': '\x05ʶ\x7f'}
4---- <class 'list'> [{'pt': 'functor', 'name': 'hacl', 'module': None, 'args': [{'pt': 'atom', 'name': 'subnet1_host_1'}, {'pt': 'atom', 'name': 'subnet1_host_1'}, {'pt': 'variable', 'name': '\x06ʶ\x7f'}, {'pt': 'variable', 'name': '\x06ʶ\x7f'}]}, {'pt': 'functor', 'name': 'hacl', 'module': None, 'args': [{'pt': 'atom', 'name': 'subnet1_host_1'}, {'pt': 'atom', 'name': 'subnet1_host_2'}, {'pt': 'variable', 'name': '\x08\x07ʶ\x7f'}, {'pt': 'va

In [ ]:
# primitives
# primitive(inCompetent(_principal)).
# primitive(competent(_principal)).
# primitive(clientProgram(_host, _programname)).
# primitive(vulExists(_host, _vulID, _program)).
# primitive(vulProperty(_vulID, _range, _consequence)).
# primitive(hacl(_src, _dst, _prot, _port)).
# primitive(attackerLocated(_host)).
# primitive(hasAccount(_principal, _host, _account)).
# primitive(networkServiceInfo(_host, _program, _protocol, _port, _user)).
# primitive(setuidProgramInfo(_host, _program, _owner)).
# primitive(nfsExportInfo(_server, _path, _access, _client)).
# primitive(nfsMounted(_client, _clientpath, _server, _serverpath, _access)).
# primitive(localFileProtection(_host, _user, _access, _path)).
# primitive(dependsOn(_h, _program, _library)).
# primitive(installed(_h, _program)).
# primitive(bugHyp(_,_,_,_)).
# primitive(vulExists(_machine,_vulID,_program,_range,_consequence)).
# primitive(canAccessFile(_host, _user, _access, _path)).
# primitive(isWebServer(_host)).
# meta(cvss(_vulID, _ac)).
inCompetent = 'inCompetent(A)'
competent = 'competent(A) '
clientProgram = 'clientProgram(A, B,)'
vulExists = 'vulExists(A, B, C)'
vulProperty = 'vulProperty(A, B, C)'
hacl = 'hacl(A, B, C, D)'
attackerLocated = 'attackerLocated(A)'
hasAccount = 'hasAccount(A, B, C)'
networkServiceInfo = 'networkServiceInfo(A, B, C, D, E)'
setuidProgramInfo = 'setuidProgramInfo(A, B, C)'
nfsExportInfo = 'nfsExportInfo(A, B, C, D)'
nfsMounted = 'nfsMounted(A, B, C, D, E)'
localFileProtection = 'localFileProtection(A, B, C, D)'
dependsOn = 'dependsOn(A, B, C)'
installed = 'installed(A, B)'
bugHyp = 'bugHyp(A, B, C)'
vulExists = 'vulExists(A, B, C, D, E)'
canAccessFile = 'canAccessFile(A, B, C, D)'
isWebServer = 'isWebServer(A)'
cvss = 'cvss(A, B,)'

primitive = [inCompetent, competent, clientProgram, vulExists,vulProperty,hacl,attackerLocated,hasAccount,networkServiceInfo,
             setuidProgramInfo,nfsExportInfo,nfsMounted,localFileProtection,dependsOn,installed,bugHyp,vulExists,canAccessFile,isWebServer,cvss]


# derived 
# execCode/2.
# netAccess/3.
# canAccessHost/1.
# canAccessFile/4.
# accessFile/3.
# principalCompromised/1.
# vulExists/5.
# logInService/3.
execCode = 'execCode(A,B)'
netaccess = 'netAccess(A,B,C)'
canAccessHost = 'canAccessHost(A)'
canAccessFile = 'canAccessFile(A,B,C,D)'
accessFile = 'accessFile(A,B,C)'
accessMaliciousInput = 'accessMaliciousInput(A,B,C)'
principalCompromised = 'principalCompromised(A)'
dos = 'dos(A)'
vulExists = 'vulExists(A,B,C,D,E)'
logInService = 'logInService(A,B,C)'
advances = 'advances(A,B)'
# attackGoal = 'attackGoal(A)'

derived = [execCode, netaccess, canAccessHost, canAccessFile, dos,accessFile,accessMaliciousInput,principalCompromised,vulExists,logInService,advances,]


for q in primitive:
    name = q[:q.index("(")]
    rows = pyxsb_query('{}.'.format(q))
    print('------{}-------'.format(name))
    for row in rows:
        print(row)
        items =(item for item in row)
        print(list(items))

for q in derived:
    name = q[:q.index("(")]
    rows = pyxsb_query('{}.'.format(q))
    print('------{}-------'.format(name))
    for row in rows:
        print(row)
        items =(item for item in row)
        print(list(items))

print(x)

### Run Mulval here

In [16]:
pyxsb_command('[run].')

In [17]:
# listing = pyxsb_query('listing(hacl).')
# print(type(listing), listing)
# [print(type(i), i) for i in listing]


In [18]:
%%bash
pwd

/tmp/mulpy/runs/1dde0947


In [19]:
# primitives
# primitive(inCompetent(_principal)).
# primitive(competent(_principal)).
# primitive(clientProgram(_host, _programname)).
# primitive(vulExists(_host, _vulID, _program)).
# primitive(vulProperty(_vulID, _range, _consequence)).
# primitive(hacl(_src, _dst, _prot, _port)).
# primitive(attackerLocated(_host)).
# primitive(hasAccount(_principal, _host, _account)).
# primitive(networkServiceInfo(_host, _program, _protocol, _port, _user)).
# primitive(setuidProgramInfo(_host, _program, _owner)).
# primitive(nfsExportInfo(_server, _path, _access, _client)).
# primitive(nfsMounted(_client, _clientpath, _server, _serverpath, _access)).
# primitive(localFileProtection(_host, _user, _access, _path)).
# primitive(dependsOn(_h, _program, _library)).
# primitive(installed(_h, _program)).
# primitive(bugHyp(_,_,_,_)).
# primitive(vulExists(_machine,_vulID,_program,_range,_consequence)).
# primitive(canAccessFile(_host, _user, _access, _path)).
# primitive(isWebServer(_host)).
# meta(cvss(_vulID, _ac)).
inCompetent = 'inCompetent(A)'
competent = 'competent(A) '
clientProgram = 'clientProgram(A, B,)'
vulExists = 'vulExists(A, B, C)'
vulProperty = 'vulProperty(A, B, C)'
hacl = 'hacl(A, B, C, D)'
attackerLocated = 'attackerLocated(A)'
hasAccount = 'hasAccount(A, B, C)'
networkServiceInfo = 'networkServiceInfo(A, B, C, D, E)'
setuidProgramInfo = 'setuidProgramInfo(A, B, C)'
nfsExportInfo = 'nfsExportInfo(A, B, C, D)'
nfsMounted = 'nfsMounted(A, B, C, D, E)'
localFileProtection = 'localFileProtection(A, B, C, D)'
dependsOn = 'dependsOn(A, B, C)'
installed = 'installed(A, B)'
bugHyp = 'bugHyp(A, B, C)'
vulExists = 'vulExists(A, B, C, D, E)'
canAccessFile = 'canAccessFile(A, B, C, D)'
isWebServer = 'isWebServer(A)'
cvss = 'cvss(A, B,)'

primitive = [inCompetent, competent, clientProgram, vulExists,vulProperty,hacl,attackerLocated,hasAccount,networkServiceInfo,
             setuidProgramInfo,nfsExportInfo,nfsMounted,localFileProtection,dependsOn,installed,bugHyp,vulExists,canAccessFile,isWebServer,cvss]


# derived 
# execCode/2.
# netAccess/3.
# canAccessHost/1.
# canAccessFile/4.
# accessFile/3.
# principalCompromised/1.
# vulExists/5.
# logInService/3.
execCode = 'execCode(A,B)'
netaccess = 'netAccess(A,B,C)'
canAccessHost = 'canAccessHost(A)'
canAccessFile = 'canAccessFile(A,B,C,D)'
accessFile = 'accessFile(A,B,C)'
accessMaliciousInput = 'accessMaliciousInput(A,B,C)'
principalCompromised = 'principalCompromised(A)'
dos = 'dos(A)'
vulExists = 'vulExists(A,B,C,D,E)'
logInService = 'logInService(A,B,C)'
advances = 'advances(A,B)'
# attackGoal = 'attackGoal(A)'

derived = [execCode, netaccess, canAccessHost, canAccessFile, dos,accessFile,accessMaliciousInput,principalCompromised,vulExists,logInService,advances,]


            
    


# print(x)

### Query post run facts here

In [20]:
# import binascii
# import hashlib

# def parseRow(row):
#     results=[]
# #     resultTypes = []
#     for item in row:
# #         results.append(item)
# #         resultTypes.append(type(item))
#         if type(item) == XSBAtom:
#             results.append(item.name)
#         elif type(item) == XSBVariable:
# #             print(type(item.name), len(item.name), item.name, hashlib.md5(item.name.encode()).hexdigest())
# #             print(binascii.hexlify(bytearray(item.name)).decode('ascii'))
# #             print(binascii.hexlify(bytes(item.name, 'utf-8')))
#             results.append((binascii.hexlify(bytes(item.name, 'utf-8'))))
# #             results.append('_'.join(('var',bytes(item.name, 'utf-8'))))
#         else:
#             results.append(item)
# #     print(results)
# #     print(resultTypes)
    
#     return results

# allfacts = {}
# for q in primitive:
#     name = q[:q.index("(")]
#     rows = pyxsb_query('{}.'.format(q))
# #     print('------{}-------'.format(name))
#     result = []
# #     print(name, len(rows))
#     for row in rows:
# #         print(row)
#         items = parseRow(row)
#         result.append(items)
# #         print
# #         items =(item for item in row)
# #         print(items)
# #     print(allfacts.keys())
#     if name not in allfacts.keys():
#         allfacts[name] = []
# #     else:
#     allfacts[name].append(result)

# for q in derived:
#     name = q[:q.index("(")]
#     rows = pyxsb_query('{}.'.format(q))
# #     print('------{}-------'.format(name))
#     result = []
# #     print(name, len(rows))
#     for row in rows:
#         print(row)
#         items = parseRow(row)
# #         items =(item for item in row)
#         result.append(items)
# #         print(items)
# #     print(allfacts.keys())
#     if name not in allfacts.keys():
#         allfacts[name] = []
# #     else:
#     if allfacts[name] is not None:
#         allfacts[name].append(result)

# allfacts

In [21]:

# q = 'listing()'
# rows = pyxsb_query('{}.'.format(q))
# print(len(rows), rows)
# for row in rows:
#     print(row)


# print('--------------------')
# for row in pyxsb_query('listing().'):
# #     print(u"hacl %s %s %s %s" % (row[0], row[1], row[2], row[3]))
#     print(row[0])
# print('--------------------')
# for row in pyxsb_query('vulProperty(A,B,C).'):
#     print(u"vulprop %s %s %s" % (row[0], row[1], row[2]))

# print('---------attackGoal-----------')
# print(pyxsb_query('findall(attackGoal(A), attackGoal(A), L), list_apply(L,write_clause_to_stdout).'))

# print('---------attackerLocated-----------')
# print(pyxsb_query('findall(attackerLocated(A), attackerLocated(A), L), list_apply(L,write_clause_to_stdout).'))
    
# print('---------vulProperty-----------')
# print(pyxsb_query('findall(vulProperty(A,B,C),vulProperty(A,B,C),L),list_apply(L,write_clause_to_stdout).'))

# print('----------vulExists----------')
# print(pyxsb_query('findall(vulExists(A,B,C),vulExists(A,B,C),L),list_apply(L,write_clause_to_stdout).'))

# print('---------cvss-----------')
# print(pyxsb_query('findall(cvss(CVE, AC),cvss(CVE, AC),L),list_apply(L,write_clause_to_stdout).'))

# print('----------hacl----------')
# print(pyxsb_query('findall(listing(),listing(),L),list_apply(L,write_clause_to_stdout).'))

In [22]:
print('here')

here


In [23]:
# # primitives
# # primitive(inCompetent(_principal)).
# # primitive(competent(_principal)).
# # primitive(clientProgram(_host, _programname)).
# # primitive(vulExists(_host, _vulID, _program)).
# # primitive(vulProperty(_vulID, _range, _consequence)).
# # primitive(hacl(_src, _dst, _prot, _port)).
# # primitive(attackerLocated(_host)).
# # primitive(hasAccount(_principal, _host, _account)).
# # primitive(networkServiceInfo(_host, _program, _protocol, _port, _user)).
# # primitive(setuidProgramInfo(_host, _program, _owner)).
# # primitive(nfsExportInfo(_server, _path, _access, _client)).
# # primitive(nfsMounted(_client, _clientpath, _server, _serverpath, _access)).
# # primitive(localFileProtection(_host, _user, _access, _path)).
# # primitive(dependsOn(_h, _program, _library)).
# # primitive(installed(_h, _program)).
# # primitive(bugHyp(_,_,_,_)).
# # primitive(vulExists(_machine,_vulID,_program,_range,_consequence)).
# # primitive(canAccessFile(_host, _user, _access, _path)).
# # primitive(isWebServer(_host)).
# # meta(cvss(_vulID, _ac)).
# inCompetent = 'inCompetent(A)'
# competent = 'competent(A) '
# clientProgram = 'clientProgram(A, B,)'
# vulExists = 'vulExists(A, B, C)'
# vulProperty = 'vulProperty(A, B, C)'
# hacl = 'hacl(A, B, C, D)'
# attackerLocated = 'attackerLocated(A)'
# hasAccount = 'hasAccount(A, B, C)'
# networkServiceInfo = 'networkServiceInfo(A, B, C, D, E)'
# setuidProgramInfo = 'setuidProgramInfo(A, B, C)'
# nfsExportInfo = 'nfsExportInfo(A, B, C, D)'
# nfsMounted = 'nfsMounted(A, B, C, D, E)'
# localFileProtection = 'localFileProtection(A, B, C, D)'
# dependsOn = 'dependsOn(A, B, C)'
# installed = 'installed(A, B)'
# bugHyp = 'bugHyp(A, B, C)'
# vulExists = 'vulExists(A, B, C, D, E)'
# canAccessFile = 'canAccessFile(A, B, C, D)'
# isWebServer = 'isWebServer(A)'
# cvss = 'cvss(A, B,)'

# primitive = [inCompetent, competent, clientProgram, vulExists,vulProperty,hacl,attackerLocated,hasAccount,networkServiceInfo,
#              setuidProgramInfo,nfsExportInfo,nfsMounted,localFileProtection,dependsOn,installed,bugHyp,vulExists,canAccessFile,isWebServer,cvss]


# # derived 
# # execCode/2.
# # netAccess/3.
# # canAccessHost/1.
# # canAccessFile/4.
# # accessFile/3.
# # principalCompromised/1.
# # vulExists/5.
# # logInService/3.
# execCode = 'execCode(A,B)'
# netaccess = 'netAccess(A,B,C)'
# canAccessHost = 'canAccessHost(A)'
# canAccessFile = 'canAccessFile(A,B,C,D)'
# accessFile = 'accessFile(A,B,C)'
# accessMaliciousInput = 'accessMaliciousInput(A,B,C)'
# principalCompromised = 'principalCompromised(A)'
# dos = 'dos(A)'
# vulExists = 'vulExists(A,B,C,D,E)'
# logInService = 'logInService(A,B,C)'
# advances = 'advances(A,B)'
# # attackGoal = 'attackGoal(A)'

# derived = [execCode, netaccess, canAccessHost, canAccessFile, dos,accessFile,accessMaliciousInput,principalCompromised,vulExists,logInService,advances,]


# for q in primitive:
#     name = q[:q.index("(")]
#     rows = pyxsb_query('{}.'.format(q))
#     print(name)
#     for row in rows:
#         items =(item for item in row)
#         print(list(items))

# for q in derived:
#     name = q[:q.index("(")]
#     rows = pyxsb_query('{}.'.format(q))
#     print(name)
#     for row in rows:
#         print(row[0])

# print(x)

In [24]:
# for row in pyxsb_query('netAccess(A, B, C).'):
#     print(u"netAccess %s %s %s" % (row[0], row[1], row[2]))
    
# for row in pyxsb_query('advances(A,B).'):
#     print(u"advances %s %s " % (row[0], row[1]) )
    
# for row in pyxsb_query('canAccessHost(A).'):
#     print(u"canAccessHost %s " % (row[0], ) )
    
# for row in pyxsb_query('hacl(A, B, C, D).'):
#     print(u"hacl %s %s %s %s" % (row[0], row[1], row[2], row[3]))

In [25]:
%%bash
pwd

/tmp/mulpy/runs/1dde0947


## MulVal Input Network Transformer

We need a representation of our network model as it is available to attack graph generation. So, just before we run mulval,  let's dump a .dot representation of current knowledge base. 